In [39]:
import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import networkx as nx
import matplotlib.pyplot as plt


Cutomer NER training 

In [40]:
model = 'en_core_web_lg'
n_iter=100

In [41]:
if model is not None:
    nlp = spacy.load('en_core_web_lg')  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Loaded model 'en_core_web_lg'


In [42]:
TRAIN_DATA= [("The foolish thief had cut the stick shorter by two inches as he thought it would grow two inches ",{"entities":[(12,17,"Person-CHAR"),(30,35,"PROD"),(47,57,"QUANTITY"),(86,96,"QUANTITY")]}),
("He knew who the thief was.",{"entities":[(16,21,"Person-CHAR")]}),
("The next day, Birbal summoned the servants again.",{"entities":[(4,12,"DAY"),(14,20,"Person-Name"),(34,42,"Person-CHAR")]}),
("On hearing this, Birbal thought of a clever plan and summoned the merchants servants.",{"entities":[(17,23,"Person-Name"),(66,75,"Person-CHAR"),(76,84,"Person-CHAR")]}),
("He suspected that one of his servants had robbed from him.",{"entities":[(29,37,"Person-CHAR"),(42,48,"EVENT")]}),
("Once, a wealthy merchant came to King Akbar's court to seek help from Birbal.",{"entities":[(38,43,"Person-Name"),(16,24,"Person-CHAR"),(33,37,"Person-CHAR"),(46,51,"LOC"),(70,76,"Person-Name")]}),
("The leader of the mice met with the elephants and requested them to take a different route to the lake. ",{"entities":[(18,22,"Animal-CHAR"),(36,45,"Animal-CHAR"),(98,102,"LOC")]}),
("The elephants would have to cross the village to go to the lake ",{"entities":[(4,13,"Animal-CHAR"),(38,45,"LOC"),(59,63,"LOC")]}),
("As the crocodile carried the monkey on his back across the river, he told him about his wife's plan to eat his heart.",{"entities":[(7,16,"Animal-CHAR"),(29,35,"Animal-CHAR"),(59,64,"LOC")]}),
("His wife, a wicked crocodile, enjoyed the sweet berries, but then told her husband that she wanted to eat the monkeys heart as that would be sweeter!",{"entities":[(4,8,"Person-CHAR"),(19,28,"Animal-CHAR"),(42,55,"VEG"),(75,82,"Person-CHAR"),(110,117,"Animal-CHAR")]}),
("The crocodile thanked the monkey ",{"entities":[(4,13,"Animal-CHAR"),(26,32,"Animal-CHAR")]}),
("He gave the crocodile some berries ",{"entities":[(12,21,"Animal-CHAR"),(27,34,"VEG")]}),
("the giant was fearsome in his looks",{"entities":[(4,9,"Person-CHAR")]}),       
("since the door and window was closed, the room was dark",{"entities":[(10,14,"PROD"),(19,25,"PROD"),(42,46,"LOC")]}),             
("Once, he saw a crocodile rest under the tree who looked tired and hungry.",{"entities":[(40,44,"PROD"),(15,24,"Animal-CHAR")]}),
("There was a monkey that lived on a berry tree on the riverbank. He ate the delicious berries every day.",{"entities":[(12,18,"Animal-CHAR"),(35,45,"PROD"),(53,62,"LOC"),(85,92,"VEG")]}),
("The second pig put in a little effort and built a house made of sticks.",{"entities":[(11,14,"Animal-CHAR"),(50,55,"LOC"),(64,70,"PROD")]}),
("The first pig barely put in any effort and built a house made of straw.",{"entities":[(10,13,"Animal-CHAR"),(65,70,"PROD"),(51,56,"LOC")]}),
("Once, there lived three little pigs who were sent out into the world by their mother.",{"entities":[(0,4,"TIME"),(31,35,"Person-CHAR"),(63,68,"LOC"),(78,84,"Person-CHAR")]}),
("Midas went home to his wife and daughter touching pebbles, rocks, and plants on the way, which turned into gold.",{"entities":[(0,5,"Person-Name"),(23,27,"Person-CHAR"),(32,40,"Person-CHAR"),(50,57,"PROD"),(59,64,"PROD"),(70,76,"PROD"),(107,111,"PROD")]}),
("Midas helped the Satyr and asked for his wish to be granted in return.",{"entities":[(0,5,"Person-Name"),(17,22,"Person-Name")]}),
("One day, Midas saw a Satyr (an angel ) who was stuck and was in trouble",{"entities":[(0,7,"DATE"),(9,14,"Person-Name"),(31,36,"Person-CHAR")]}),
("He was extremely wealthy and had all the gold he could ever need. He also had a daughter whom he loved very much.",{"entities":[(41,45,"NAT-RES"),(80,88,"Person-CHAR")]}),
("A long time ago, there lived a king in Greece named Midas.",{"entities":[(2,15,"TIME"),(31,35,"Person-CHAR"),(39,45,"LOC"),(52,57,"Person-Name")]}),
("But, the villagers assumed that he was pulling a silly prank again and did not come to his rescue. ",{"entities":[(9,18,"Person-CHAR")]}),
("On hearing his cries, the villagers rushed to help him chase the wolf away and safe the sheep.",{"entities":[(26,35,"Person-CHAR"),(65,69,"Animal-CHAR"),(88,93,"Animal-CHAR")]}),
("One day, the boy was extremely bored as he watched over the sheep and so he cried: Wolf! Wolf!",{"entities":[(0,7,"DAY"),(13,16,"Person-CHAR"),(60,65,"Animal-CHAR"),(83,87,"Animal-CHAR"),(89,93,"Animal-CHAR")]}),
("undefined",{"entities":[(28,34,"Person-CHAR"),(38,44,"Person-CHAR"),(82,87,"Animal-CHAR"),(88,95,"PROCESS"),(96,105,"TIME")]}),
("They were kept tied up with weak rope that they couldve easily escaped, but did not.",{"entities":[(33,37,"PROD")]}),
("Once upon a time in a circus, five elephants that performed circus tricks.",{"entities":[(22,28,"LOC"),(35,44,"Animal-CHAR"),(60,66,"LOC")]}),
("She dreamt of buying a cake, a basket of strawberries, a fancy dress, and even a new house with the money she would make selling the eggs and the milk",{"entities":[(23,27,"PROD"),(31,37,"PROD"),(41,53,"VEG"),(57,68,"PROD"),(85,90,"PROD"),(133,137,"PROD"),(146,150,"PROD")]}),
("She thought of buying a hen and selling its eggs and she planned on becoming wealthy.",{"entities":[(24,27,"Animal-CHAR"),(44,48,"PROD")]}),
("As she was walking to the market, she began to daydream about what she would do from the money she fo for the milk.",{"entities":[(26,32,"LOC"),(110,114,"PROD")]}),
("She milked her cow and carried the two pails of milk she fetched on a stick and set out to sell the milk at the market.",{"entities":[(4,10,"PROCESS"),(15,19,"Animal-CHAR"),(48,52,"PROD"),(70,75,"PROD"),(100,104,"PROD"),(112,118,"LOC")]}),
("There was once a milkmaid named Patty.",{"entities":[(17,25,"Person-CHAR"),(32,37,"Person-Name")]}),
("While the hare was asleep, the tortoise crossed the finish line and won the race!",{"entities":[(10,14,"Animal-CHAR"),(31,39,"Animal-CHAR"),(52,63,"LOC"),(76,80,"EVENT")]}),
("the tortoise walked slowly but extremely determined and dedicated.",{"entities":[(4,12,"Animal-CHAR")]}),
("he took a nap right near the finish line.",{"entities":[(29,40,"LOC")]}),
("A farmer planted grapes in his field ",{"entities":[(2,8,"Person-CHAR"),(17,23,"VEG"),(31,36,"LOC")]}),             
("His ego made him believe that he could win the race even if he rested for a while.",{"entities":[(47,51,"EVENT")]}),
("The tortoise agreed and the race began.",{"entities":[(3,13,"Animal-CHAR"),(28,32,"EVENT")]}),
("The hare got a head-start but became overconfident towards the end of the race.",{"entities":[(4,8,"Animal-CHAR"),(74,78,"EVENT")]}),
("The tortoise agreed and the race began.",{"entities":[(4,12,"Animal-CHAR")]}),
("The hare simply wanted to prove that he was the best and have the satisfaction of beating him.",{"entities":[(4,8,"Animal-CHAR")]}),
("The story began when the hare who has won many races proposed a race with the tortoise.",{"entities":[(25,29,"Animal-CHAR"),(78,86,"Animal-CHAR")]}),
("The other frogs were shocked at this and wondered how he did it.",{"entities":[(10,15,"Animal-CHAR")]}),
("The other frog keeps trying and eventually jumps so high that he escapes the pit",{"entities":[(10,14,"Animal-CHAR"),(77,80,"LOC")]}),
("They continued to constantly discourage them as the two frogs tried to jump out of the pit",{"entities":[(56,61,"Animal-CHAR"),(87,90,"LOC")]}),
("The other frogs worried about their friends in the pit.",{"entities":[(10,15,"Animal-CHAR"),(51,54,"LOC")]}),
("Alice sat under a tree",{"entities":[(0,5,"Person-Name"),(18,22,"PROD")]}),
("wit is useless components",{"entities":[(0,3,"PROD")]}),
("it took a couple of weeks for him to get back to his senses.",{"entities":[(10,25,"DATE")]}),
("the price of gold and diamonds are on the rise",{"entities":[(13,17,"NAT-RES"),(22,30,"NAT-RES")]}),             
("There was a hole under the tree near the cave",{"entities":[(27,31,"PROD"),(41,44,"LOC")]}),
("Suddenly, two frogs in the group accidentally fell into a deep pit",{"entities":[(14,19,"Animal-CHAR"),(63,66,"LOC")]}),
("John noticed that the potato had become soft and its skin was peeling off easily; the egg had become harder and tougher; the coffee beans had completely changed and filled the bowl of water with aroma and flavour",{"entities":[(22,28,"VEG"),(86,89,"VEG"),(125,137,"PROD"),(176,180,"PROD"),(184,189,"NAT-RES")]}),
("Once the bowls had cooled down, Johns father asked him to feel the texture of the different food items again.",{"entities":[(9,14,"PROD"),(32,37,"Person-Name"),(38,44,"Person-CHAR"),(92,96,"VEG")]}),
("Rajeev lived in a kingdome and loved to eat watermelon ",{"entities":[(0,6,"Person-Name"),(18,26,"LOC"),(44,54,"VEG")]}),
("John did as he had been told, father then boiled all three bowls",{"entities":[(0,4,"Person-Name"),(30,36,"Person-CHAR"),(59,64,"PROD")]}),
("He then asked John to feel their texture and then fill each bowl with water.",{"entities":[(70,75,"NAT-RES"),(14,19,"Person-Name"),(60,64,"PROD")]}),
("His father simply smiled and asked him to get a potato, an egg, and some coffee beans. He placed them in three bowls.",{"entities":[(4,10,"Person-CHAR"),(48,54,"VEG"),(59,62,"PROD"),(73,85,"PROD"),(111,116,"PROD")]}),
("When his father asked John why he was crying, he said that he had a lot of problems in his life.",{"entities":[(9,15,"Person-CHAR"),(22,26,"Person-Name")]}),
("A boy named John was upset. His father found him crying.",{"entities":[(12,16,"Person-Name"),(32,38,"Person-CHAR")]}),
]

In [43]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(factory_name='ner', last=True)
else:
    ner = nlp.get_pipe('ner')

In [ ]:
from spacy.training.example import Example

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.create_optimizer()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update(
                [example],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)
        print(itn)

100%|██████████| 63/63 [00:02<00:00, 24.72it/s]


{'ner': 286.17116658094034}
0


100%|██████████| 63/63 [00:02<00:00, 25.42it/s]


{'ner': 215.7369714266201}
1


100%|██████████| 63/63 [00:02<00:00, 24.47it/s]


...

100%|██████████| 63/63 [00:02<00:00, 25.69it/s]


{'ner': 9.765687276040838}
69


100%|██████████| 63/63 [00:02<00:00, 25.61it/s]


{'ner': 14.479710025548881}
70


 10%|▉         | 6/63 [00:00<00:02, 25.47it/s]

In [ ]:
output_dir = Path('C:\\Users\\ashwa\custom_ner_model2')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)
nlp.to_disk('custom_ner_model2')
